In [ ]:
import sys
import pandas as pd

## Building

In [ ]:
building = pd.read_csv("../../data/raw/building_metadata.csv")

In [ ]:
building

In [ ]:
building.dtypes

In [ ]:
building.count()

In [ ]:
building = building.astype({'building_id': "int32", 'site_id': 'int32'})

In [ ]:
for c in building.columns:
    if building[c].dtype == "float":
        building[c] = building[c].astype("float32")

In [ ]:
building.to_feather("../../data/raw/building_metadata.feather")

## Weather

In [ ]:
weather = pd.read_csv("../../data/raw/weather.csv")

In [ ]:
weather.dtypes

In [ ]:
weather["timestamp"] = pd.to_datetime(
            weather["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [ ]:
def to_float32(df):
    for c in df.columns:
        if df[c].dtype == "float":
            df[c] = df[c].astype("float32")
    return df

In [ ]:
weather = to_float32(weather)

In [ ]:
weather = weather.astype({'site_id': 'int32'})

In [ ]:
weather.to_feather("../../data/raw/weather.feather")

In [ ]:
weather.dtypes

## Meters

In [ ]:
meters = pd.read_csv("../../data/raw/meters.csv")

In [ ]:
meters.shape

In [ ]:
meters = meters.sample(frac=0.1, random_state=0)

In [ ]:
meters["timestamp"] = pd.to_datetime(
            meters["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [ ]:
meters = meters[meters.building_id != 778]

In [ ]:
meters.shape

In [ ]:
meters.dtypes

In [ ]:
meters = meters.astype({'building_id': 'int32', "meter": "int32", "meter_reading": "float32"})

In [ ]:
meters = meters.reset_index(drop=True)

In [ ]:
meters.dtypes

In [ ]:
meters.to_feather("../../data/raw/meters.feather")

## Merge

In [ ]:
data_dir = "/home/jovyan/personal_workspace/bootcamp/data"

In [ ]:
from tqdm import notebook
import os

In [ ]:
FILES_FOLDER = {
    'raw': ['raw/building_metadata.feather', 'raw/meters.feather', 'raw/weather.feather'],
    'clean': ['clean/building_metadata.feather', 'clean/meters.feather', 'clean/weather.feather'],
    'merged': ['merged/data.feather'],
    'model': ['model/train.feather', 'model/val.feather', 'model/test.feather']
}


def load_data(folder, data_dir, dict_files=FILES_FOLDER):
    files = dict_files[folder]

    dataframes = []

    print(":: Start loading data")
    for name_file in notebook.tqdm(files):
        dataframe = pd.read_feather(os.path.join(data_dir, name_file))
        dataframes.append(dataframe)

    return dataframes

In [ ]:
building, meters, weather = load_data("raw", data_dir)

In [ ]:
building

In [ ]:
meters.dtypes

In [ ]:
weather

In [ ]:
dataset = meters.merge(building, how='left', on='building_id')

In [ ]:
dataset = dataset.merge(
    weather,
    how='left',
    on=['site_id', 'timestamp']
)

In [ ]:
dataset

In [ ]:
dataset.count()